<a href="https://colab.research.google.com/github/MDankloff/ScriptAIRbnb.R/blob/main/testCOMPAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix

In [26]:
#using a simplified subset from COMPAS obtained from kaggle (Ofer, 2017)
cdata = pd.read_csv("/content/propublica_data_for_fairml.csv")
cdata.head()


,Two_yr_Recidivism,Number_of_Priors,score_factor,Age_Above_FourtyFive,Age_Below_TwentyFive,African_American,Asian,Hispanic,Native_American,Other,Female,Misdemeanor
0,0,0,0,1,0,0,0,0,0,1,0,0
1,1,0,0,0,0,1,0,0,0,0,0,0
2,1,4,0,0,1,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,1
4,1,14,1,0,0,0,0,0,0,0,0,0


In [35]:
target = "Two_yr_Recidivism"
X = cdata.drop([target], axis= 1)
Y = cdata[target]


In [50]:
len(cdata)

6172

In [36]:
#split test train set - stratify=y ensures that the distribution of classes in the target variable y is maintained in both the training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.2, shuffle =True, stratify = Y)

In [43]:
#train random forest classifier
rf = RandomForestClassifier()
rf.fit(X_train, Y_train)

RandomForestClassifier()

In [49]:
#predict
Y_hat = rf.predict(X_test)
len(Y_hat)

1235

In [54]:
#store predicted and true classes in a data frame
#create empty data frame
Outcome = pd.DataFrame()

# Add column named 'predicted_class' to Outcome with values from Y_hat, converted to a list using
Outcome ['predicted_class'] = Y_hat.tolist()

# Sets the index of Outcome to be the same as the index of X_test. This aligns the predicted classes with the corresponding samples in the test set.
Outcome = Outcome.set_index(X_test.index)

#Add column named 'true_class' to Outcome with the true classes from y_test
Outcome['true_class'] = Y_test